# Projeto Sidra

## Business Undestanding

## Data Undestanding

## Data Preparation

### Setup

In [1]:
import pandas as pd
import requests
import sqlite3
import re
from urllib.parse import urlparse

### Extração dos dados

In [2]:
def extrair(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

### Tratamento dos dados

In [27]:
def transformar(json_data):
    # Verifica o nome correto das colunas e os registros
    nome_colunas = json_data[0]
    mapeamento_colunas = {k: v for k, v in nome_colunas.items()}
    dados = json_data[1:] 

    # Cria o DataFrame
    df = pd.DataFrame(dados)
    df = df.rename(columns=mapeamento_colunas)

    # Tratamento da coluna 'Valor'
    if 'Valor' in df.columns:
        df['Valor'] = df['Valor'].replace('-', None)
        df['Valor'] = df['Valor'].str.replace('.', '', regex=False)
        df['Valor'] = df['Valor'].str.replace(',', '.', regex=False)
        df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')

    # Remover colunas duplicadas e colunas com '(Código)'
    df = df.drop_duplicates()
    colunas_a_excluir = [col for col in df.columns if '(Código)' in col]
    df = df.drop(columns=colunas_a_excluir, errors='ignore')
   
    # Remover valores faltantes na coluna 'Valor'
    df = df.dropna(subset=['Valor'])

    # Preencher valores faltantes com 'Não informado'
    df = df.fillna('Não informado')

    # Feature: extrair ano se houver coluna de Mês
    if 'Mês' in df.columns:
        df['Ano'] = df['Mês'].str.extract(r'(\d{4})')

    # Exclui os valores 'Totais' contidos na coluna 'Produto das lavouras'
    df = df[~df['Produto das lavouras'].str.contains('Total', na=False)]

    # Exclui os índices dos valores contidos na coluna 'Produto das lavouras'
    df['Produto das lavouras'] = df['Produto das lavouras'].str.replace(r'^\d+(\.\d+)*\s+', '', regex=True)

    return df

### Load dos dados

In [4]:
# Função para extrair nomes das tabelas
def extrair_codigo_tabela(url):
    match = re.search(r'/t/(\d+)', url)
    return f"t{match.group(1)}" if match else "tabela_desconhecida"

# Função para fazer o load no banco de dados
def load(df, nome_tabela, nome_banco):
    conexao = sqlite3.connect(nome_banco)
    df.to_sql(nome_tabela, conexao, if_exists='replace', index=False)
    conexao.close()
    print(f"✅ Tabela '{nome_tabela}' salva em '{nome_banco}'.")

### Execução do código

In [5]:
def executar_etl(url):
    print(f"\n🚀 Processando URL: {url}")
    json_dados = extrair(url)
    df = transformar(json_dados)

    nome_base = extrair_codigo_tabela(url)
    nome_arquivo_banco = f"sidra_{nome_base}.db"
    nome_tabela = nome_base

    load(df, nome_tabela=nome_tabela, nome_banco=nome_arquivo_banco)

In [24]:
executar_etl('https://apisidra.ibge.gov.br/values/t/1618/n3/all/v/all/p/all/c49/all/c48/all')


🚀 Processando URL: https://apisidra.ibge.gov.br/values/t/1618/n3/all/v/all/p/all/c49/all/c48/all
✅ Tabela 't1618' salva em 'sidra_t1618.db'.


In [28]:
dados = extrair('https://apisidra.ibge.gov.br/values/t/1618/n3/all/v/all/p/all/c49/all/c48/all')
df = transformar(dados)

In [29]:
df.head()

,Nível Territorial,Unidade de Medida,Valor,Unidade da Federação,Variável,Mês,Ano da safra,Produto das lavouras,Ano
1,Unidade da Federação,Hectares,1047805.0,Rondônia,Área plantada,abril 2025,Safra 2024,"Cereais, leguminosas e oleaginosas",2025
2,Unidade da Federação,Hectares,9862.0,Rondônia,Área plantada,abril 2025,Safra 2024,Algodão herbáceo,2025
5,Unidade da Federação,Hectares,41809.0,Rondônia,Área plantada,abril 2025,Safra 2024,Arroz,2025
9,Unidade da Federação,Hectares,2649.0,Rondônia,Área plantada,abril 2025,Safra 2024,Feijão (1ª Safra),2025
10,Unidade da Federação,Hectares,153.0,Rondônia,Área plantada,abril 2025,Safra 2024,Feijão (2ª Safra),2025


In [22]:
df_tratado = df[~df['Produto das lavouras'].str.contains('Total', na=False)]
df_tratado.head()

,Nível Territorial,Unidade de Medida,Valor,Unidade da Federação,Variável,Mês,Ano da safra,Produto das lavouras,Ano
0,Unidade da Federação,Hectares,1047805.0,Rondônia,Área plantada,abril 2025,Safra 2024,"1 Cereais, leguminosas e oleaginosas",2025
1,Unidade da Federação,Hectares,9862.0,Rondônia,Área plantada,abril 2025,Safra 2024,1.1 Algodão herbáceo,2025
4,Unidade da Federação,Hectares,41809.0,Rondônia,Área plantada,abril 2025,Safra 2024,1.4 Arroz,2025
8,Unidade da Federação,Hectares,2649.0,Rondônia,Área plantada,abril 2025,Safra 2024,1.8 Feijão (1ª Safra),2025
9,Unidade da Federação,Hectares,153.0,Rondônia,Área plantada,abril 2025,Safra 2024,1.9 Feijão (2ª Safra),2025


In [23]:
df_tratado['Produto das lavouras'] = df_tratado['Produto das lavouras'].str.replace(r'^\d+(\.\d+)*\s+', '', regex=True)
df_tratado.head()

C:\Users\Mitsuya\AppData\Local\Temp\ipykernel_8884\873356520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tratado['Produto das lavouras'] = df_tratado['Produto das lavouras'].str.replace(r'^\d+(\.\d+)*\s+', '', regex=True)


,Nível Territorial,Unidade de Medida,Valor,Unidade da Federação,Variável,Mês,Ano da safra,Produto das lavouras,Ano
0,Unidade da Federação,Hectares,1047805.0,Rondônia,Área plantada,abril 2025,Safra 2024,"Cereais, leguminosas e oleaginosas",2025
1,Unidade da Federação,Hectares,9862.0,Rondônia,Área plantada,abril 2025,Safra 2024,Algodão herbáceo,2025
4,Unidade da Federação,Hectares,41809.0,Rondônia,Área plantada,abril 2025,Safra 2024,Arroz,2025
8,Unidade da Federação,Hectares,2649.0,Rondônia,Área plantada,abril 2025,Safra 2024,Feijão (1ª Safra),2025
9,Unidade da Federação,Hectares,153.0,Rondônia,Área plantada,abril 2025,Safra 2024,Feijão (2ª Safra),2025
